<a href="https://colab.research.google.com/github/JonathanLehner/Colab-collection/blob/master/video_to_bvh.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# needs python 2
# https://github.com/Dene33/video_to_bvh/blob/master/video_to_bvh.ipynb

##Initial imports, install, initializations 
####Be patient it'll take some time :)

In [2]:
!git clone https://github.com/Dene33/keras_Realtime_Multi-Person_Pose_Estimation.git
!pip install configobj
!apt install ffmpeg
!apt-get install blender

!apt install python2.7 python-pip
!git clone https://github.com/Dene33/hmr.git
!wget https://people.eecs.berkeley.edu/~kanazawa/cachedir/hmr/models.tar.gz && tar -xf models.tar.gz
!mv models hmr/
!pip2 install -r hmr/requirements.txt
!mkdir hmr/output
!mkdir hmr/output/csv
!mkdir hmr/output/images
!mkdir hmr/output/csv_joined
!mkdir hmr/output/bvh_animation

import os
os.chdir('keras_Realtime_Multi-Person_Pose_Estimation')
!bash model/get_keras_model.sh
!mkdir sample_jsons
!mkdir sample_videos
!mkdir sample_images

exec(open('model_load.py').read())
!pip2 install opendr==0.77

Cloning into 'keras_Realtime_Multi-Person_Pose_Estimation'...
remote: Enumerating objects: 196, done.
remote: Total 196 (delta 0), reused 0 (delta 0), pack-reused 196
Receiving objects: 100% (196/196), 29.49 MiB | 22.14 MiB/s, done.
Resolving deltas: 100% (80/80), done.
  Stored in directory: /root/.cache/pip/wheels/f1/e4/16/4981ca97c2d65106b49861e0b35e2660695be7219a2d351ee0
Successfully built configobj
Reading package lists... Done
Building dependency tree       
Reading state information... Done
ffmpeg is already the newest version (7:3.4.6-0ubuntu0.18.04.1).
The following package was automatically installed and is no longer required:
  libnvidia-common-410
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 16 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-410
Use 'apt autoremove' to remove it.
Th

Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.
     |████████████████████████████████| 245kB 9.3MB/s 
  Stored in directory: /root/.cache/pip/wheels/5c/cd/7c/8f4035d040aacef26bccc11cebfe69cc44871e020b1f5e598b
Successfully built opendr


##Upload video

In [3]:
#upload video
exec(open('upload_videos.py').read())

In [17]:
%cd /content/
!cp -r hmr keras_Realtime_Multi-Person_Pose_Estimation/hmr

/content/keras_Realtime_Multi-Person_Pose_Estimation


##Process the video
####<font color='green'>`24`</font> in `video_to_images.sh` stands for desired fps rate at which you want to convert video to images

In [20]:
%cd /content/keras_Realtime_Multi-Person_Pose_Estimation/
# keras_Realtime_Multi-Person_Pose_Estimation/

#convert to images, specify fps rate
!bash video_to_images.sh 24

#2d pose estimation. For each image creates corresponding .json file with format
#similar to output .json format of openpose (https://github.com/CMU-Perceptual-Computing-Lab/openpose)
exec(open('2d_pose_estimation.py').read())

#3d pose estimation
os.chdir('..')
!bash hmr/3dpose_estimate.sh

#convert estimated .csv files to bvh
!blender --background hmr/csv_to_bvh.blend -noaudio -P hmr/csv_to_bvh.py

/content/keras_Realtime_Multi-Person_Pose_Estimation
Processing *
ffmpeg version 3.4.6-0ubuntu0.18.04.1 Copyright (c) 2000-2019 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.3.0-16ubuntu3)
  configuration: --prefix=/usr --extra-version=0ubuntu0.18.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enabl

##Download .bvh

In [5]:
from google.colab import files
files.download('hmr/output/bvh_animation/estimated_animation.bvh')

FileNotFoundError: ignored

##Display results of 3d pose estimation (optional)

In [0]:
from IPython.display import Image, display

images = os.listdir('hmr/output/images')
extensions = {".jpg", ".png", ".gif", ".bmp", ".jpeg"}

for image in sorted(images):
  for ext in extensions:
    if image.endswith(ext):
      display(Image('hmr/output/images/'+image, width=512))

##Clear all the generated data if you want to process new video

In [0]:
!rm keras_Realtime_Multi-Person_Pose_Estimation/sample_images/*
!rm keras_Realtime_Multi-Person_Pose_Estimation/sample_jsons/*
!rm keras_Realtime_Multi-Person_Pose_Estimation/sample_videos/*
!rm hmr/output/bvh_animation/*
!rm hmr/output/csv/*
!rm hmr/output/csv_joined/*
!rm hmr/output/images/*

####Based on:


1.   **End-to-end Recovery of Human Shape and Pose** by *Angjoo Kanazawa, Michael J. Black, David W. Jacobs, Jitendra Malik *
https://github.com/akanazawa/hmr  

  Corresponding fork by *Denis Cera*: https://github.com/Dene33/hmr

2.   **Realtime Multi-Person 2D Pose Estimation using Part Affinity Fields** by *Zhe Cao, Tomas Simon, Shih-En Wei, Yaser Sheikh* \
Keras implementation by *Michal Faber*: https://github.com/michalfaber/keras_Realtime_Multi-Person_Pose_Estimation

  Corresponding fork by *Denis Cera*: https://github.com/Dene33/keras_Realtime_Multi-Person_Pose_Estimation


